## Quick Start: Training an IMDb sentiment model with *ULMFiT*

Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps:

1. Reading and viewing the IMDb data
1. Getting your data ready for modeling
1. Fine-tuning a language model
1. Building a classifier

In [1]:
from fastai.text import * 

Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do is to preprocess our data so that we change the raw texts to lists of words, or tokens (a step that is called tokenization) then transform these tokens into numbers (a step that is called numericalization). These numbers are then passed to embedding layers that will convert them in arrays of floats before passing them through a model.

Steps:

1. Get your data preprocessed and ready to use,
1. Create a language model with pretrained weights that you can fine-tune to your dataset,
1. Create other models such as classifiers on top of the encoder of the language model.

To show examples, we have provided a small sample of the [IMDB dataset](https://www.imdb.com/interfaces/) which contains 1,000 reviews of movies with labels (positive or negative).

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

In [3]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


#### Get the data and create a `databunch` object to use for a language model

In [4]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # The following line throws `AttributeError: backwards` on the learning step, below
        # data_lm = TextDataBunch.from_csv(path, 'texts.csv')
        # This Fastai Forum post shows the solution:
        #      https://forums.fast.ai/t/backwards-attributes-not-found-in-nlp-text-learner/51340?u=jcatanza
        # We implement the solution on the following line:
        data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

failure count is 0

CPU times: user 457 ms, sys: 227 ms, total: 684 ms
Wall time: 3.07 s


#### Get the data again, this time form a `databunch` object for use in a classifier model

In [5]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Create the databunch for the classifier model
        data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')


failure count is 0

CPU times: user 721 ms, sys: 229 ms, total: 950 ms
Wall time: 3.15 s


#### Save the `databunch` objects for later use

In [6]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

#### Load the `databunch` objects
Note that you can load the data with different [`DataBunch`](/basic_data.html#DataBunch) parameters (batch size, `bptt`,...)

In [7]:
#bs=192
#bs=48
bs=32

In [8]:
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

### 1. Build and fine-tune a language model

We can use the `data_lm` object we created earlier to fine-tune a pretrained language model. [fast.ai](http://www.fast.ai/) has an English model with an AWD-LSTM architecture available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.

#### Set up to use the GPU

In [9]:
# torch.cuda.set_device(1)
torch.cuda.set_device(0)

#### Build the IMDb language model, initializing with the hpretrained weights from `wikitext-103`

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

#### Start training
By default this step unfreezes and trains the weights for the last layer. Training updates the weights to values more applicable to the language of IMDb reviews.

In [11]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.299138,3.914893,0.280446,00:09


In [12]:
language_model_learner??

In [ ]:
learn.load_pretrained??

In [ ]:
convert_weights??

You can use [Visual Studio Code](https://code.visualstudio.com/) (vscode - open source editor that comes with recent versions of Anaconda, or can be installed separately), or most editors and IDEs, to browse code. vscode things to know:

- Command palette (<kbd>Ctrl-shift-p</kbd>)
- Go to symbol (<kbd>Ctrl-t</kbd>)
- Find references (<kbd>Shift-F12</kbd>)
- Go to definition (<kbd>F12</kbd>)
- Go back (<kbd>alt-left</kbd>)
- View documentation
- Hide sidebar (<kbd>Ctrl-b</kbd>)
- Zen mode (<kbd>Ctrl-k,z</kbd>)

#### Unfreeze the weights and train some more

Like a computer vision model, we can then unfreeze the model and fine-tune it.
In this step we are allowing the model to update *all* the weights with values more suitable to the language of IMDb reviews. But we are mindful that the pretrained weights from wikitext-103 are likely already near their optimal values. For this we train the weights in the "earlier" layers with a much lower learning rate.

In [13]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.990114,3.925152,0.280937,00:11
1,3.674947,3.901868,0.282381,00:11
2,3.109032,3.941258,0.280536,00:11


#### Test the language model

To evaluate your language model, you can run the [`Learner.predict`](/basic_train.html#Learner.predict) method and specify the number of words you want it to guess.

In [14]:
learn.predict("This is a review about", n_words=10)

'This is a review about Major Payne , showing the pivotal moment of'

In [15]:
learn.predict("In the hierarchy of horror movies this has to be near the top.", n_words=10)

"In the hierarchy of horror movies this has to be near the top. The actual gore , light and then you 're"

Sometimes the generated text doesn't make much sense because we have a tiny vocabulary and didn't train much. But note that the model respects basic grammar, which comes from the pretrained model.

Finally we save the encoder to be able to use it for classification in the next section.

#### Save the encoder

In [16]:
learn.save('mini_imdb_language_model')
learn.save_encoder('mini_imdb_language_model_encoder')

### 2. Build a movie review classifier
We use mixed precision (`.to_fp16()`)for greater speed, smaller memory footprint, and a regularizing effect.

In [17]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('mini_imdb_language_model_encoder')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj absolutely awful movie . xxmaj utter waste of time . 
 
  background music is so loud that you can not understand speech . xxmaj well if you really listen closely , whatever they speak is actually unintelligible . 
 
  xxmaj camera work is bad , editing is not present , background score gives a headache , action is shoddy , dialogs are unintelligible , xxmaj acting is abysmal and well xxmaj xxunk used to look like a wrestler , now she looks like a xxunk wrestler . xxmaj hell you can slim down but you can not gain grace . 
 
  xxmaj after spending three hours watching a movie i want to like it , but this movie would not even allow me that pleasure . 
 
  xxmaj please if you want to torture yourself , go ahead watch this .,xxbos i had read many good things about this adaptation of my favorite novel ... so invariably my expectations were crushed . xxmaj but they were crushed more than should be expect

In [18]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk xxunk experience in general . xxmaj with graphics",positive


#### Train the last layer of the classifier

In [19]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.601477,0.626736,0.606965,00:07


#### Unfreeze the classifier model and fine-tune

In [20]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.441964,0.484579,0.791045,00:11
1,0.381773,0.432320,0.776119,00:11
2,0.296648,0.372120,0.830846,00:12


#### Test the classifier
We can use our model to predict on a few example of movie review-like raw text by using the [`Learner.predict`](/basic_train.html#Learner.predict) method. 

Our model is 70% sure that this is a positive review.

In [21]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

(Category tensor(0), tensor(0), tensor([0.6514, 0.3486]))

Our model is 83% sure that this is a positive review:

In [22]:
learn.predict('Not so good World War II epic film')

(Category tensor(1), tensor(1), tensor([0.0560, 0.9440]))

Bottom line: the model did not do a good job, misclassifying both reviews (with high confidence, to boot!)

#### Let's train some more, and re-try these examples to see if we can do better

In [23]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.250756,0.421156,0.815920,00:12
1,0.224827,0.450621,0.796020,00:12
2,0.167750,0.360277,0.850746,00:12


#### Re-try the above examples 

In [30]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
  warn('Tensor is int32: upgrading to int64; for better performance use int64 input')


(Category negative, tensor(0), tensor([0.9879, 0.0121]))

In [31]:
learn.predict('Not so good World War II epic film')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
  warn('Tensor is int32: upgrading to int64; for better performance use int64 input')


(Category negative, tensor(0), tensor([0.7353, 0.2647]))

The model seems to have benefitted from the extra training, since it now correctly classifies both reviews as `negative`, with high confidence.

#### Make up your own text in the style of a movie review. Then use our classifier to see what it thinks!